In [2]:
import os
import glob
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau

from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [3]:
train_dir = '../input/chest-xray-covid19-pneumonia/Data/train/*'
test_dir = '../input/chest-xray-covid19-pneumonia/Data/test/*'
train2_covid_dir = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID'

In [4]:
SIZE = 128
BATCH_SIZE = 64
TARGET_SIZE = (SIZE,SIZE)
EPOCH_NUM = 200

In [5]:
categories_dict = {
  0: "PNEUMONIA",
  1: "NORMAL",
  2: "COVID19"
}

In [6]:
img=cv2.imread('../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/COVID-1013.png')
plt.imshow(img)
img.shape

## Extracting subbands using DWT + Merging subbands in one image with shape(128,128,4)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pywt
import pywt.data


# Load image
original = cv2.resize(img,(256,256)) 
original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
# Wavelet transform of image, and plot approximation and details
titles = ['Approximation', ' Horizontal detail',
      'Vertical detail', 'Diagonal detail']
coeffs2 = pywt.dwt2(original, 'bior1.3')
LL, (LH, HL, HH) = coeffs2
fig = plt.figure(figsize=(12, 3))
for i, a in enumerate([LL, LH, HL, HH]):
  ax = fig.add_subplot(1, 4, i + 1)
  ax.imshow(a, interpolation="nearest", cmap=plt.cm.gray)
  ax.set_title(titles[i], fontsize=10)
  ax.set_xticks([])
  ax.set_yticks([])

fig.tight_layout()
plt.show()

In [8]:
## Merging 
merged = cv2.merge([LL, LH, HL, HH])
print(merged.shape)

## Loading Train and Test Data

In [9]:
train_images = []
train_labels = [] 
label = 0

#Importing the first training dataset

for directory_path in glob.glob(train_dir):
    assert categories_dict[label] == os.path.normpath(directory_path).split(os.path.sep)[-1]
    print(categories_dict[label])
    counter = 1
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        if(counter%200==0): print(counter,"images loaded")
        img = cv2.imread(img_path, 0)
        img = cv2.resize(img, TARGET_SIZE)
        coeffs2 = pywt.dwt2(img, 'bior1.3')
        LL, (LH, HL, HH) = coeffs2
        for i in [LL,LH,HL,HH]:
            i=cv2.resize(i,TARGET_SIZE)
        img = cv2.merge([LL, LH, HL, HH])
        img=cv2.resize(img,TARGET_SIZE)
        train_images.append(img)
        train_labels.append(label)
        counter+=1
        if(counter%1500==0): break
    
    print(counter,"images loaded")
    label +=1
    
#Importing the additional training dataset

print("additional",categories_dict[2],"data")    
addit_counter = 1
for img_path in glob.glob(os.path.join(train2_covid_dir, "*.png")):
    if(addit_counter%200==0): print(addit_counter,"images loaded")
    img = cv2.imread(img_path, 0)
    img = cv2.resize(img, TARGET_SIZE)
    coeffs2 = pywt.dwt2(img, 'bior1.3')
    LL, (LH, HL, HH) = coeffs2
    for i in [LL,LH,HL,HH]:
            i=cv2.resize(i,TARGET_SIZE)
    img = cv2.merge([LL, LH, HL, HH])
    img=cv2.resize(img,TARGET_SIZE)
    train_images.append(img)
    train_labels.append(2)
    addit_counter+=1
    if(addit_counter%1000==0): break
print(addit_counter,"images loaded")

x_train = np.array(train_images)
y_train = to_categorical(train_labels, 3)

In [10]:
x_train[200].shape

In [11]:
test_images = []
test_labels = []
label = 0

#importing the testing dataset

for directory_path in glob.glob(test_dir):
    assert categories_dict[label] == os.path.normpath(directory_path).split(os.path.sep)[-1]
    print(categories_dict[label])
    counter = 1
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        if(counter%100==0): print(counter, "images loaded")
        img = cv2.imread(img_path, 0)
        img = cv2.resize(img, TARGET_SIZE)
        coeffs2 = pywt.dwt2(img, 'bior1.3')
        LL, (LH, HL, HH) = coeffs2
        for i in [LL,LH,HL,HH]:
            i=cv2.resize(i,TARGET_SIZE)
        img = cv2.merge([LL, LH, HL, HH])
        img=cv2.resize(img,TARGET_SIZE)
        test_images.append(img)
        test_labels.append(label)
        counter+=1
    
    print(counter,"images loaded")
    label +=1

test_images = np.array(test_images)
test_labels = to_categorical(test_labels, 3)

In [12]:
test_images[0].shape

### Splitting data into train and validation dataset

In [13]:
train_test_split(train_images, train_labels)
train_images, val_images, train_labels, val_labels = train_test_split(x_train, 
                                                                      y_train, 
                                                                      test_size=0.15, 
                                                                      random_state=69)

In [14]:
print("train:",train_images.shape[0],", test:",test_images.shape[0],", val:",val_images.shape[0])

## Feature Extraction from subbands using GLCM

In [15]:
from skimage import util, exposure, data
def feature_glcm(image):
    glcm_dataset = pd.DataFrame()
    [LL,LH,HL,HH]=cv2.split(image)
    df = pd.DataFrame()    
    for n ,img in enumerate([LL,LH,HL,HH]):
            img = exposure.rescale_intensity(img, out_range=(0, 1))
            bin_width = 32
            im = util.img_as_ubyte(img)
            img = im//bin_width
            GLCM = greycomatrix(img, [1], [0])       
            GLCM_Energy = greycoprops(GLCM, 'energy')[0]
            df['Energy__subband'+str(n)] = GLCM_Energy
            GLCM_corr = greycoprops(GLCM, 'correlation')[0]
            df['Corr__subband'+str(n)] = GLCM_corr       
            GLCM_diss = greycoprops(GLCM, 'dissimilarity')[0]
            df['Diss_sim__subband'+str(n)] = GLCM_diss       
            GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
            df['Homogen__subband'+str(n)] = GLCM_hom       
            GLCM_contr = greycoprops(GLCM, 'contrast')[0]
            df['Contrast__subband'+str(n)] = GLCM_contr
            GLCM_ASM = greycoprops(GLCM, 'ASM')[0]
            df['ASM__subband'+str(n)] = GLCM_ASM

    glcm_dataset = glcm_dataset.append(df)
    return glcm_dataset

In [16]:
feature_glcm(merged)

In [17]:
def feature_extractor(images):
    image_dataset = pd.DataFrame()
    for image in images: 
        image_dataset = image_dataset.append(feature_glcm(image))
        
    return image_dataset

In [18]:
train_df= feature_extractor(train_images)

In [19]:
train_df

In [20]:
test_df = feature_extractor(test_images)

In [21]:
test_df

In [22]:
val_df = feature_extractor(val_images)

In [23]:
val_df

In [24]:
# convert from integers to floats
train_images_norm = train_images.astype('float32')
test_images_norm = test_images.astype('float32')
val_images_norm = val_images.astype('float32')
# normalize to the range 0-1
train_images_norm /= 255.0
test_images_norm /= 255.0
val_images_norm /= 255.0

In [26]:
pd.set_option("display.max_columns", None)
train_df

## Define U-NET model 

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate,Flatten
from tensorflow.keras.optimizers import Adam


def unet(pretrained_weights = None,input_size = (SIZE,SIZE,4)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4) # for crop and copy
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size=(2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3) # Concatenate for localization informantion
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(4, 1, activation = 'sigmoid')(conv9)
    f=Flatten()(conv10)
    d=Dense(4, activation=tf.keras.activations.relu)(f)
    model = Model(inputs=inputs, outputs=d)
    
    model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [28]:
from keras.models import Sequential
import keras
def build_mlp():
    model = Sequential([
        keras.Input(shape=24, name='Extracted_GLCM_Features'),
        keras.layers.Dense(8, activation=tf.keras.activations.relu, name='Dense1'),
        keras.layers.Dense(4, activation=tf.keras.activations.relu, name='Dense2')
    ])
    print(model.summary())
    return model

In [29]:
mlp = build_mlp()
unet = unet()

combinedInput = concatenate([mlp.output, unet.output])

x = Dense(8, activation="relu")(combinedInput)
x = Dense(3, activation="softmax")(x)

model_u = Model(inputs=[mlp.input, unet.input], outputs=x)

In [30]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.005)

In [31]:
model_u.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

plot_model(model_u, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [32]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

cb = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=10,
        mode='auto',
        min_delta=0.0002,
        cooldown=5,
        min_lr=10e-8,
        verbose=1,
    )
]

## Learning

In [33]:
train_dfu = train_df[:500]
train_images_normu=train_images_norm[:500]
train_labelsu=train_labels[:500]
val_dfu=val_df[:100]
val_images_normu=val_images_norm[:100]
val_labelsu=val_labels[:100]
test_dfu = test_df[:200]
test_images_normu=test_images_norm[:200]
test_labelsu=test_labels[:200]

In [34]:
dataset_inputs = tf.data.Dataset.from_tensor_slices((train_dfu, tf.expand_dims(train_images_normu, axis=-1)))
dataset_label = tf.data.Dataset.from_tensor_slices(train_labelsu)

dataset = tf.data.Dataset.zip((dataset_inputs, dataset_label)).batch(BATCH_SIZE).repeat()
STEP_SIZE_TRAIN= train_images_norm.shape[0]//BATCH_SIZE
# fit model
with tf.device('/gpu:0'):
    print('runnning from gpu')
    history_u = model_u.fit(dataset, 
                        validation_data=([val_dfu, tf.expand_dims(val_images_normu, axis=-1)], val_labelsu),
                        epochs = EPOCH_NUM, steps_per_epoch=32, callbacks=cb)

In [36]:
model_u.save('unet_200itrs.h5')

<a href="./unet_200itrs.h5"> Download File </a>

In [37]:
import os 
os.chdir(r'/kaggle/working')

%cd /kaggle/working


from IPython.display import FileLink
FileLink(r'unet_200itrs.h5')

## Testing

In [38]:
test_inputs = tf.data.Dataset.from_tensor_slices((test_dfu, tf.expand_dims(test_images_normu, axis=-1)))
test_labels = tf.data.Dataset.from_tensor_slices(test_labelsu)

test_dataset = tf.data.Dataset.zip((test_inputs, test_labels)).batch(BATCH_SIZE).repeat()
STEP_SIZE_TEST= test_images_norm.shape[0]//BATCH_SIZE

score = model_u.evaluate(test_dataset, batch_size=BATCH_SIZE, steps=STEP_SIZE_TEST )
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

## Limitation des ressources (simple CNN)

In [39]:
def build_cnn():
    model = keras.Sequential([
        keras.Input(shape=(SIZE,SIZE,4), name='Original_Images'),
        keras.layers.Conv2D(input_shape=(140,140,1), filters=32, kernel_size=11, 
                            strides=1, activation='relu', name='Conv1'),
        keras.layers.Conv2D(input_shape=(130,130,32), filters=32, kernel_size=11, 
                            strides=1, activation='relu', name='Conv2'),
        keras.layers.MaxPool2D(pool_size=(5, 5), strides=2,padding='same'),
        keras.layers.Conv2D(input_shape=(58,58,32), filters=64, kernel_size=9, 
                            strides=1, activation='relu', name='Conv3'),
        keras.layers.MaxPool2D(pool_size=(5, 5), strides=2,padding='same'),
        keras.layers.Conv2D(input_shape=(23,23,64), filters=128, kernel_size=8, 
                            strides=1, activation='relu', name='Conv4'),
        keras.layers.Conv2D(input_shape=(16,16,128), filters=256, kernel_size=9, 
                            strides=1, activation='relu', name='Conv5'),
        keras.layers.Conv2D(input_shape=(8,8,256), filters=256, kernel_size=8, 
                            strides=1, activation='relu', name='Conv6'),    

        keras.layers.Flatten(),
        keras.layers.Dense(8, activation=tf.keras.activations.relu, name='Dense')
    ])
    print(model.summary())
    return model

In [40]:
def build_mlp():
    model = keras.Sequential([
        keras.Input(shape=24, name='Extracted_Traditional_Features'),
        keras.layers.Dense(8, activation=tf.keras.activations.relu, name='Dense1'),
        keras.layers.Dense(4, activation=tf.keras.activations.relu, name='Dense2')
    ])
    print(model.summary())
    return model

In [41]:
mlp = build_mlp()
cnn = build_cnn()

combinedInput = concatenate([mlp.output, cnn.output])

x = Dense(8, activation="relu")(combinedInput)
x = Dense(3, activation="softmax")(x)

model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [42]:
model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [43]:
dataset_inputs = tf.data.Dataset.from_tensor_slices((train_df, tf.expand_dims(train_images_norm, axis=-1)))
dataset_label = tf.data.Dataset.from_tensor_slices(train_labels)

dataset = tf.data.Dataset.zip((dataset_inputs, dataset_label)).batch(BATCH_SIZE).repeat()
STEP_SIZE_TRAIN= train_images_norm.shape[0]//BATCH_SIZE
# fit model
with tf.device('/gpu:0'):
    print('runnning from gpu')
    history = model.fit(dataset, 
                    validation_data=([val_df, tf.expand_dims(val_images_norm, axis=-1)], val_labels),
                    epochs = EPOCH_NUM, steps_per_epoch=STEP_SIZE_TRAIN, callbacks=cb)

In [48]:
model.save("mlp_cnn_200itrs_new.h5")

<a href="./mlp_cnn_200itrs_new.h5"> Download File </a>

## Testing

In [49]:
test_inputs = tf.data.Dataset.from_tensor_slices((test_df, tf.expand_dims(test_images_norm, axis=-1)))
# test_labels = tf.data.Dataset.from_tensor_slices(test_labels)

test_dataset = tf.data.Dataset.zip((test_inputs, test_labels)).batch(BATCH_SIZE).repeat()
STEP_SIZE_TEST= test_images_norm.shape[0]//BATCH_SIZE

score = model.evaluate(test_dataset, batch_size=BATCH_SIZE, steps=STEP_SIZE_TEST )
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [50]:
import matplotlib.pyplot as plt
history.history.keys()

In [51]:
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [52]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss curve')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()